In [ ]:
import os
import numpy as np
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

ruta = os.path.join('.', 'datos', 'PercMultAplicado.xlsx')
datos = pd.read_excel(ruta)

In [ ]:
datos

In [ ]:
# ------------------------ limpieza de datos ------------------------ #
datos.columns = datos.columns.str.lower()
datos['carga'] = datos['mensualidad'] / datos['ingreso mensual']
datos['mora_bool'] = datos.mora.replace({'SI': 1, 'NO': 0})
datos.rename(columns={'antigüedad laboral (meses)': 'antiguedad_laboral_meses'}, inplace=True)

subset = datos[['monto', 'antiguedad_laboral_meses', 'carga', 'mora_bool']]

In [ ]:
subset

# Histograma columna "carga"

In [ ]:
# graficar histograma de carga
plt.hist(subset.carga, bins=20)
plt.show()

In [ ]:
subset.carga.min()

In [ ]:
subset.carga.max()

# Histograma columna "monto"

In [ ]:
plt.hist(subset.monto, bins=20)
plt.show()

In [ ]:
# calcular raíz cuadrada de monto
subset['monto_sqrt'] = np.sqrt(subset.monto)

In [ ]:
plt.hist(subset.monto_sqrt, bins=20)
plt.show()

In [ ]:
subset.monto_sqrt.min()

In [ ]:
subset.monto_sqrt.max()

In [ ]:
# ------------------ ingeniería de características ------------------ #

# normalizar monto con min-max
subset['monto_norm'] = (subset.monto_sqrt - 100) / (600 - 100)

In [ ]:
subset

In [ ]:
subset.monto_norm.min()

In [ ]:
subset.monto_norm.max()

# Histograma columna "antiguedad_laboral_meses"

In [ ]:
plt.hist(subset.antiguedad_laboral_meses, bins=20)
plt.show()

In [ ]:
# normalizar antiguedad_laboral_meses con min-max
subset['al_norm'] = (subset.antiguedad_laboral_meses - 0) / (subset.antiguedad_laboral_meses.max() - 0)

In [ ]:
subset

In [ ]:
subset.al_norm.min()

In [ ]:
subset.al_norm.max()

In [ ]:
subset_norm = subset[['carga', 'monto_norm', 'al_norm', 'mora_bool']]
subset_norm

# Proceso de creación de función para generar los sets de entrenamiento y testeo

In [ ]:
from generacion_sets import generar_sets

# ------------------------ generación de sets ------------------------ #
train, test = generar_sets(df=subset_norm, col_bool='mora_bool', prop=0.7, prop_interna_f=0.8, prop_interna_t=0.2)

In [ ]:
train.mora_bool.value_counts()

In [ ]:
test.mora_bool.value_counts()

In [ ]:
train

In [ ]:
test

# Entrenamiento de la red neuronal

In [ ]:
# -------------------- # Preparación de los datos # ---------------------- #

# Separar en X y d
X = train[['carga', 'monto_norm', 'al_norm']].to_numpy().astype(float)
d = train['mora_bool'].to_numpy().astype(float)

In [ ]:
# ------------------------------ # Fase 1 # ------------------------------ #
# definiendo las dimensiones
Q, N = X.shape  # Q es el número de ejemplos, N es el número de entradas
L = 6 #  neuronas en la capa oculta
M = 1 #  neuronas en la capa de salida

# inicializando los pesos (de inicio, son aleatorios)
w_h = np.random.uniform(-1, 1, (L, N))
w_o = np.random.uniform(-1, 1, (M, L))

In [ ]:
print(Q, N, L, M)
print(w_h.shape)
print(w_o.shape)

In [ ]:
# -------------------- # Preparación de los datos # ---------------------- #

# función de activación (el cerebro)
def func_sigmoidea(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
# -------------------- # función de entrenamiento # ---------------------- #
def proceso_entrenamiento(x, d, w_h, w_o, alfa, precision):
    """
    Genera los pesos del perceptrón multicapa.

    Parámetros:
    ----------
    x : np.ndarray
        Matriz de entradas.
    d : np.ndarray
        Matriz de salidas deseadas.
    w_h : np.ndarray
        Matriz de pesos de la capa oculta.
    w_o : np.ndarray
        Matriz de pesos de la capa de salida.
    alfa : float
        Tasa de aprendizaje.
    precision : float
        Precisión del error.
    
    Regresa:
    --------
    w_h : np.ndarray
        Matriz de pesos de la capa oculta.
    w_o : np.ndarray
        Matriz de pesos de la capa de salida.
    """
    # Procedimiento de entrenamiento
    E = float('inf')

    while E > precision:
        E = 0
        for j in range(Q):
            # proceso forward
            net_h = np.dot(w_h, x[j].T)
            y_h = func_sigmoidea(net_h)
            net_o = np.dot(w_o, y_h)
            y = func_sigmoidea(net_o)

            # proceso backward
            delta_o = ((d[j] - y) * y * (1 - y))
            delta_h = y_h * (1 - y_h) * np.dot(w_o.T, delta_o)
            # ajuste de los pesos (w's)
            Delta_w_o = alfa * np.outer(delta_o, y_h)
            Delta_w_h = alfa * np.outer(delta_h, x[j])

            w_o = w_o + Delta_w_o
            w_h = w_h + Delta_w_h

            # Acumulando el error
            E += np.linalg.norm(delta_o)

        print(f'error: {E}')

    return w_h, w_o 

In [ ]:
# ------------------ # proceso de entrenamiento # -------------------- #
w_h, w_o = proceso_entrenamiento(X, d, w_h, w_o, alfa=0.5, precision=0.001)